In [ ]:
import os
import azureml.core
from azureml.core import Workspace, Experiment, Datastore, Dataset,Environment
from azureml.widgets import RunDetails

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

In [ ]:
from azureml.pipeline.core import Pipeline, PipelineParameter
from azureml.pipeline.steps import PythonScriptStep
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig


print("Pipeline SDK-specific imports completed")

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

In [ ]:
from azureml.core.compute import AmlCompute
aml_compute = AmlCompute(ws, "gpucluster")

In [ ]:
from azureml.core.runconfig import RunConfiguration
run_config = RunConfiguration()
run_config.environment.docker.enabled = True
env = Environment.get(ws, "pytorch-env")
run_config.environment = env

In [ ]:
dataset = Dataset.get_by_name(ws, name='cifar10')
ds_input = dataset.as_named_input('input1')

In [ ]:

# Uses default values for PythonScriptStep construct.

source_directory = './code/pytorch-cloud'
print('Source directory for the step is {}.'.format(os.path.realpath(source_directory)))

step1 = PythonScriptStep(name="train_step",
                         script_name="train.py",
                         arguments=[
                            '--data_path', ds_input.as_download(),
                            '--learning_rate', 0.003,
                            '--momentum', 0.92],
                         #inputs=[ds_input],
                         compute_target=aml_compute, 
                         source_directory=source_directory,
                         runconfig=run_config,
                         allow_reuse=True)
print("Step1 created")

In [ ]:
steps = [step1]

In [ ]:
pipeline1 = Pipeline(workspace=ws, steps=steps)
print ("Pipeline is built")

In [ ]:
pipeline1.validate()

In [ ]:
pipeline1.submit("pipeline-run")